In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 11 08:41:55 2024

@author: Manasa Kesapragada
"""

import pandas as pd
import numpy as np
import math
from keras import backend as K
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers.legacy import Adam
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
import random
import tensorflow as tf
import seaborn as sns
import pdb
from keras.regularizers import l2

Initialize Data

In [29]:
def initialize_data(df_train, df_test, input_cols):
  columns_to_drop = ['X_POSITION', 'Y_POSITION']
  df_train = df_train.drop(columns=columns_to_drop)
  df_test = df_test.drop(columns=columns_to_drop)
  #df['drug'] = df['drug'].fillna(0)

  # Reset particle IDs to start from 1
  df_train['new_spore_id'], unique_ids = pd.factorize(df_train['SPORE_ID'])
  df_train['new_spore_id'] = df_train['new_spore_id'] + 1

  df_test['new_spore_id'], unique_ids = pd.factorize(df_test['SPORE_ID'])
  df_test['new_spore_id'] = df_test['new_spore_id'] + 1

  # Drop index columns
  df_train = df_train.drop(['Unnamed: 0'], axis=1)
  df_test = df_test.drop(['Unnamed: 0'], axis=1)
  scaler = MinMaxScaler(feature_range = (0,1))
  
  df_train["INTENSITY"] = scaler.fit_transform(df_train[["INTENSITY"]])
  df_test["INTENSITY"] = scaler.fit_transform(df_test[["INTENSITY"]])

  return df_train, df_test

### Model

In [30]:
def create_dataset(df, lookback, in_cols, cells, drug_conc=[1]):
    out_cols=["GERMINATION"]
    trainX, trainY = [], [] #lists of training and testing inputs/outputs
    for drug in drug_conc:
        for track in range(cells[0], cells[1]):
            cell = df.loc[(df["new_spore_id"] == track)] #all rows of data pertaining to this cell
            cell = cell[in_cols] #reduce it to our columns of interest
            for i in range(len(cell)-lookback-1):
                trainX.append(cell[i:i+lookback])
            cell = cell[out_cols]
            #pdb.set_trace()
            for i in range(len(cell)-lookback-1):
                trainY.append(cell[i+lookback+1:i+lookback+2])

    trainX = np.array(list(map(lambda x: x.to_numpy(), trainX)))
    trainY = np.array(list(map(lambda x: x.to_numpy(), trainY)))
    return np.array(trainX), np.array(trainY)

def split_data(df_train, df_test, lookback, input_cols):
    # calculate data for train with 80% val with 20%
    num_train_data = len(list(df_train["SPORE_ID"].unique()))
    train_len = int(num_train_data * 0.85)
    num_test_data = len(df_test["SPORE_ID"].unique())
    print(f"training on spores (1, {train_len})...")
    print(f"validating on spores ({train_len + 1}, {num_train_data})...")
    print(f"testing on spores ({1, num_test_data})...")

    #training
    trainX, trainY = create_dataset(df_train, lookback, input_cols, cells=(1, train_len))
    valX, valY = create_dataset(df_train, lookback, input_cols, cells=(train_len + 1, num_train_data))
    #testing
    testX, testY = create_dataset(df_test, lookback, input_cols, cells=(1, num_test_data))

    return trainX, trainY, valX, valY, testX, testY

In [31]:
#trains numerous models using a list of numbers to initialize the models
def typical_model(experiment_type, trainX,trainY,valX,valY,testX,testY, numbers, model_file = None):
    models = [] #list of models
    predictions = [] #list of prediction vectors

    for i in numbers:
        
        # MODEL TRAINING
        if model_file == None: 
            print('Training model number {}'.format(i))
            model = Sequential()
            model.add(LSTM(80, input_shape=(trainX.shape[1], trainX.shape[2]))) 
            model.add(Dense(1, activation='sigmoid'))
            model.add(Dropout(0.01))
            model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
            
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            
            history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=30, batch_size=1, verbose=1, callbacks=[early_stopping])
            models.append(model)
            model.summary()

            model.save(f"{experiment_type}_initial_{i}.h5")
            print("Saved model {} to disk".format(i))

            # PLOT TRAINING AND VALIDATION LOSS
            loss = history.history['loss']
            val_loss = history.history['val_loss']
            epochs = range(1, len(loss) + 1)
            
            plt.figure()
            plt.plot(epochs, loss, 'bo', label='Training loss')
            plt.plot(epochs, val_loss, 'b', label='Validation loss')
            plt.title('Training and validation loss for model {}'.format(i))
            plt.legend()
            plt.show()
            
            # PLOT TRAINING AND VALIDATION ACCURACY
            loss = history.history['accuracy']
            val_loss = history.history['val_accuracy']
            epochs = range(1, len(loss) + 1)
            
            plt.figure()
            plt.plot(epochs, loss, 'bo', label='Training accuracy')
            plt.plot(epochs, val_loss, 'b', label='Validation accuracy')
            plt.title('Training and validation accuracy for model {}'.format(i))
            plt.legend()
            plt.show()        
        
        if model_file != None: 
            print("loading model...")
            model = load_model(model_file)
            models.append(model)


        #Predict on training, validation, and test sets
        trainPredict = model.predict(trainX)
        valPredict = model.predict(valX)
        testPredict = model.predict(testX)

        # Threshold the predictions to get binary outputs
        trainPredict_binary = (trainPredict > 0.5).astype(int)
        valPredict_binary = (valPredict > 0.5).astype(int)
        testPredict_binary = (testPredict > 0.5).astype(int)


        # Calculate RMSEs
        trainScore = math.sqrt(mean_squared_error(trainY[:,:, 0], trainPredict_binary))
        print('Training RMSE: {}'.format(trainScore))
        valScore = math.sqrt(mean_squared_error(valY[:,:, 0], valPredict_binary))
        print('Validation RMSE: {}'.format(valScore))
        testScore = math.sqrt(mean_squared_error(testY[:,:, 0], testPredict_binary))
        print('Testing RMSE: {}'.format(testScore))

        # add predictions to list of prediction vectors
        preds = np.concatenate([trainPredict_binary, valPredict_binary, testPredict_binary], axis=None)
        predictions.append(preds)


    #find the average of all prediction vectors
    mean_pred = np.mean(predictions,axis=0)

    #find the prediction vector that is closest to the mean
    closest = 0 #index of the prediction vector that is closest to the mean
    dist = 100 #the distance of that closest vector to the mean vector
    for i in range(len(predictions)):
        thisdist = math.sqrt(mean_squared_error(predictions[i], mean_pred))

        print('Model number: {}, distance from mean: {}'.format(i,thisdist))

        if thisdist < dist:
            dist = thisdist
            closest = i

    #return the "most average" model
    print('Returning model {}, whose distance is {}'.format(closest,dist))
    return models, closest


#models, closest = typical_model(trainX,trainY,valX,valY,testX,testY,range(1), model_file)
#model = models[closest]
#model.save("alex_data_model_lk2.h5")
#model.save("alex_data_model_lk2.keras")
#print("Saved model to disk")


### Sensitivity Analysis

In [52]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

def local_sensitivity_analysis(df, features: str, std_factor: int):
    original_data = df.copy()
    delta_xs = []
    original_color = "blue"
    perturbed_color = "orange"
    for feature in features:
        print(f"perturbing {feature.title()}...")
        # Calculate mean and standard deviation
        feature_stddev: float = np.std(df[feature])
        feature_mean: float = np.mean(df[feature])

        x_min = np.min(df[feature]) - feature_stddev
        x_max = np.max(df[feature]) + feature_stddev
        
        #print(rf"{feature.title()} has std = {feature_stddev}...")
        #print(rf"{feature.title()} has mean = {feature_mean}...")

        std = feature_stddev * std_factor

        ### Adding Noise
        #print(f"Using std = {std} to create Gaussian noise...")
        noise = np.random.normal(0, std, df[feature].shape)
        #print(f"Adding {df[feature].shape} Gaussian noise samples from {np.min(noise)} to {np.max(noise)}...")
        df[feature] = df[feature] + noise

        delta_x = np.mean(np.abs(original_data[feature] - df[feature]))
        delta_xs.append(delta_x)
        

    # fig, ax = plt.subplots(1, 2, figsize=(14, 4))  # Changed to 2 subplots for distribution plots

    # ax[1].hist(original_data[feature], bins=30, alpha=0.25, color=original_color, label = "Original")
    # ax[1].hist(df[feature], bins=30, alpha=0.25, color=perturbed_color, label = "Perturbed")
    # ax[1].set_title(f"{feature.title()} Distribution")
    # ax[1].set_xlabel('Value')
    # ax[1].set_ylabel('Frequency')
    # ax[1].set_xlim([x_min, x_max])
    # ax[1].legend(loc="best")

    # # Plot: Noise distribution plot
    # ax[0].axvline(x=std, linestyle="--", color='lightgrey', label=rf"+$\sigma$")
    # ax[0].axvline(x=-std, linestyle="--", color='lightgrey', label=rf"-$\sigma$")
    # ax[0].hist(noise, bins=30, alpha=0.75, color='skyblue', edgecolor="black")
    # ax[0].set_title(rf"Noise Distribution with $\sigma$={std:.2f}")
    # ax[0].set_xlabel('Noise Value')
    # ax[0].set_ylabel('Frequency')
    # plt.legend()
    # plt.show()

    # #Plot: Feature before and after perturbation
    # example_spore_perturbed = df[df["new_spore_id"] == 1]
    # example_spore_original = original_data[original_data["new_spore_id"]==1]
    # germination_status = example_spore_original["GERMINATION"].to_list()
    # example_germination = germination_status.index(1)    
    # plt.figure(figsize=(6, 4))

    # sns.lineplot(x=range(0, len(germination_status)), y=example_spore_original[feature], color=original_color, label="Original")
    # sns.lineplot(x=range(0, len(germination_status)), y=example_spore_perturbed[feature], color=perturbed_color, label="Perturbed", alpha = 0.5)
    
    # plt.axvline(example_germination, color='lightgrey', linestyle='--', label="Germination")
    # plt.xlabel("Frame")
    # plt.ylabel(feature.title())
    # plt.title(rf"{feature.title()} of Spore 1 with $\sigma$ = {std_factor}")
    # plt.legend(loc="best")
    # plt.show()

    

    return df, delta_xs

### Temporal Sensitivity Analysis

In [33]:
def temporal_sensitivity_analysis(df, features, time_range: [int, int], sigma):
  df_copy = df.copy()

  time_range_str = str(time_range)
  unique_spore_ids = df["new_spore_id"].unique()


  noise_allfeatures = [] 
  for feature in features: 
      feature_std = np.std(df[feature])
      feature_std_scaled = feature_std * sigma
      noise_feature = np.random.normal(0, feature_std_scaled, (abs(time_range[1] - time_range[0]) * len(unique_spore_ids)))
      noise_allfeatures.append(noise_feature)
  noise_index = 0
  germination_times = df[df["GERMINATION"] == 1].groupby('new_spore_id')['FRAME'].min().reset_index()

  for spore_id in unique_spore_ids:
      spore_data = df_copy[df_copy['new_spore_id'] == spore_id]
      spore_germination_frame = germination_times.loc[germination_times['new_spore_id'] == spore_id, 'FRAME'].values[0]

      for frame_index in range(len(spore_data)):
          frame = spore_data["FRAME"].iloc[frame_index]
          if time_range[0] < 0: 
              time_range = [spore_germination_frame + time_range[0], spore_germination_frame + time_range[1]]
          if frame in range(time_range[0], time_range[1]):
              for idx, feature in enumerate(features):
                df_copy.loc[spore_data.index[frame_index], feature] += noise_allfeatures[idx][noise_index]
              noise_index += 1

  example_original = df[df["new_spore_id"] == 1]
  example_perturbed = df_copy[df_copy["new_spore_id"] == 1]

  
  time_range_str = time_range_str.replace(" ", "")
  time_range_str = time_range_str.replace("[", "")
  time_range_str = time_range_str.replace("]", "")
  time_range_str = time_range_str.split(",")

  if int(time_range_str[0]) == 0:
      time_range_str = f"First {time_range_str[1]}"
  elif int(time_range_str[1]) == 0:
      time_range_str = f"Last {abs(int(time_range_str[0]))}"
  for feature in features:
      plt.plot(example_original["FRAME"], example_original[feature], label='Original', color="blue")
      plt.plot(example_perturbed["FRAME"], example_perturbed[feature], label='Perturbed', alpha=0.5, color = "orange")
      plt.legend()
      plt.xlabel("Frame")
      plt.ylabel(f"{feature.title()}")
      plt.title(rf"{feature.title()} with {sigma}$\sigma_{feature[0]}$ at {time_range_str} Frames")
      plt.show()

  return df_copy




### Predict on Test Data

In [34]:
def predict_test_data(model, trainX, valX, testX):
  #Predict on training, validation, and test sets
  trainPredict = model.predict(trainX)
  trainPredict =(trainPredict > 0.5).astype(int)

  valPredict = model.predict(valX)
  valPredict =(valPredict > 0.5).astype(int)

  testPredict = model.predict(testX)
  testPredict =(testPredict > 0.5).astype(int)
  return trainPredict, valPredict, testPredict

### Model Performance

$ \text{True Positive Rate} = \frac{\text{TP}}{\text{TP} + \text{FN}} $

$ \text{False Positive Rate} = \frac{\text{FP}}{\text{FP} + \text{TN}} $



$ \text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}} $

$ \text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}} $

$ \text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}} $

$ \text{F1 Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} $


In [35]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def accuracy_precision_recall_f1score(trainY, trainPredict, valY, valPredict, testY, testPredict):
  # Calculate accuracy, precision, recall, and F1-score
  train_accuracy = accuracy_score(trainY, trainPredict)
  val_accuracy = accuracy_score(valY, valPredict)
  test_accuracy = accuracy_score(testY, testPredict)
  print("--------------------")
  print("Accuracy:")
  print('Training Accuracy: {}'.format(train_accuracy))
  print('Validation Accuracy: {}'.format(val_accuracy))
  print('Testing Accuracy: {}'.format(test_accuracy))

  train_precision = precision_score(trainY, trainPredict)
  val_precision = precision_score(valY, valPredict)
  test_precision = precision_score(testY, testPredict)
  print("--------------------")
  print("Precision:")
  print('Training Precision: {}'.format(train_precision))
  print('Validation Precision: {}'.format(val_precision))
  print('Testing Precision: {}'.format(test_precision))

  train_recall = recall_score(trainY, trainPredict)
  val_recall = recall_score(valY, valPredict)
  test_recall = recall_score(testY, testPredict)
  print("--------------------")
  print("Recall:")
  print('Training Recall: {}'.format(train_recall))
  print('Validation Recall: {}'.format(val_recall))
  print('Testing Recall: {}'.format(test_recall))

  train_f1 = f1_score(trainY, trainPredict)
  val_f1 = f1_score(valY, valPredict)
  test_f1 = f1_score(testY, testPredict)
  print("--------------------")
  print("F1-Score:")
  print('Training F1-Score: {}'.format(train_f1))
  print('Validation F1-Score: {}'.format(val_f1))
  print('Testing F1-Score: {}'.format(test_f1))

In [36]:
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

# Plot ROC curve
def plot_roc_curve(y_true, y_pred, title):
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_score = roc_auc_score(y_true, y_pred)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (area = {auc_score:0.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()


In [37]:
# Training set
# train_auc = roc_auc_score(trainY, trainPredict)
# print('Training AUC: {}'.format(train_auc))
# plot_roc_curve(trainY, trainPredict, 'ROC curve - Training set')

In [38]:
# Validation set
# val_auc = roc_auc_score(valY, valPredict)
# print('Validation AUC: {}'.format(val_auc))
# plot_roc_curve(valY, valPredict, 'ROC curve - Validation set')

In [39]:
# # Test set
# test_auc = roc_auc_score(testY, testPredict)
# print('Testing AUC: {}'.format(test_auc))
# plot_roc_curve(testY, testPredict, 'ROC curve - Testing set')

               Predicted
             |   0   |   1   |
    --------------------------
    True  0  |  TN   |  FP   |
    True  1  |  FN   |  TP   |


In [40]:
# Confusion matrices
# train_cm = confusion_matrix(trainY, trainPredict)
# val_cm = confusion_matrix(valY, valPredict)
# test_cm = confusion_matrix(testY, testPredict)
# print('Training Confusion Matrix:')
# print(train_cm)
# print('Validation Confusion Matrix:')
# print(val_cm)
# print('Testing Confusion Matrix:')
# print(test_cm)

In [41]:
def model_prediction_results(model, df_test, input_cols):
    print('Writing results for model {}'.format(1))
    maxtrack = int(max(df_test['new_spore_id']))
    for track in range(1, maxtrack+1):
        print(f"track {track} of {maxtrack}...")
        cell = df_test.loc[(df_test['new_spore_id']==track)]
        if len(cell)==0:
            continue
        maxslice = max(df_test.loc[(df_test['new_spore_id']==track), 'FRAME'])+1
        minslice = min(df_test.loc[(df_test['new_spore_id']==track), 'FRAME'])
        for sl in range(int(minslice),int(maxslice+1)):
            x = cell.loc[(cell['FRAME']>sl-1) & (cell['FRAME']<=sl)]
            x = x[input_cols].to_numpy()
            if x.size > 0:
                x=x.reshape(1, 1, len(input_cols))
                df_test.loc[(df_test['new_spore_id']==track) & (df_test['FRAME']==sl), 'pred_germ{}'.format(1)] = model.predict(x)
        
    df_test['pred_error{}'.format(1)] = df_test['pred_germ{}'.format(1)] - df_test['GERMINATION']
    df_test['pred_germ1'] =(df_test['pred_germ1'] > 0.5).astype(int)
    return df_test


### Plots

Plot of Predicted Spores Germinated against Actual Values

In [42]:
def plot_population_prediction(df_test):
  plt.errorbar(df_test['FRAME'].unique(),df_test.groupby(['FRAME']).mean()['pred_germ1'],
              color = 'b', label = 'Predicted values')
  plt.errorbar(df_test['FRAME'].unique(),df_test.groupby(['FRAME']).mean()['GERMINATION'],
              color = 'orange', label = 'Original values')
  plt.yticks(fontsize=20)
  plt.xlabel('Timestep',fontsize=14)
  plt.ylabel('Mean spores germinated',fontsize=14)
  plt.legend()

In [43]:
# Get unique spore IDs
def plot_individual_predictions(df_test, show_plot):
    unique_spore_ids = df_test['new_spore_id'].unique()

    # list of correct spores
    spore_error: list[0, 1] = []
    actual_germination: list[int] = []
    predicted_germination: list[int] = []
    # Loop through each spore ID and create separate plots
    for spore_id in unique_spore_ids:
        spore_data = df_test[df_test['new_spore_id'] == spore_id]
        
        if show_plot == 1:
            fig, ax = plt.subplots(figsize=(10, 5))
            
            # Plot GERMINATION values
            ax.errorbar(
                spore_data['FRAME'].unique(),
                spore_data.groupby(['FRAME']).mean()['GERMINATION'],
                yerr=spore_data.groupby(['FRAME']).sem()['GERMINATION'],
                color='red',
                linewidth = 5,
                label='Original GERMINATION values'
            )
            
            # Plot pred_germ1 values
            ax.errorbar(
                spore_data['FRAME'].unique(),
                spore_data.groupby(['FRAME']).mean()['pred_germ1'],
                yerr=spore_data.groupby(['FRAME']).sem()['pred_germ1'],
                color='black',
                linewidth = 3,
                label='Predicted GERMINATION values'
            )

            # Set plot title and labels
            ax.set_title(f'Spore ID {spore_id} - GERMINATION')
            ax.set_xlabel('FRAME')
            ax.set_ylabel('Values')
            ax.legend()

            actual_germination_frame = spore_data[spore_data['GERMINATION'] == 1]['FRAME'].min()
            actual_germination.append(actual_germination_frame)
            predicted_germination_frame = spore_data[spore_data['pred_germ1'] == 1]['FRAME'].min()
            predicted_germination.append(predicted_germination_frame)


            # Adjust layout
            plt.tight_layout()
            plt.show()


                #add 1 for correct and 0 for wrong
        binary_error = int((spore_data['GERMINATION'].mean() == spore_data['pred_germ1'].mean()))
        if binary_error == 0:
            print(f"spore {spore_id} predicted incorrectly...")
        spore_error.append(binary_error)
        
    return spore_error, actual_germination, predicted_germination


### Model Analysis

Sensitivity Analysis

In [44]:
def write_sensitivity_analysis_results(base, experiment_type, scale_factor, sensitivity_feature, delta_x, correct_percentage):
    print(f"writing sensitivity analysis results...")

    sensitivity_csv = f"{base}/GerminationPrediction/Model_Analysis/{experiment_type}_sensitivity_featurestd.csv"
    std_cell = scale_factor
    accuracy = {
        "Feature": [sensitivity_feature],
        "Accuracy": [correct_percentage],
        "STD": [std_cell],
        "Delta x": [delta_x]
        }

    accuracy_df = pd.DataFrame(accuracy)
    accuracy_df.to_csv(sensitivity_csv, index=False, mode="a", header=False)
    print(f"results written to {sensitivity_csv}...")

Temporal Sensitivity Analysis

In [45]:
def write_temporal_sensitivity_results(base, experiment_type, temporal_sensitivity_ft, correct_percentage, temporal_sensitivity_timerange, temporal_std_scaler):
  print(f"writing temporal sensitivity analysis results...")
  temporal_sensitivity_csv = f"{base}/GerminationPrediction/Model_Analysis/{experiment_type}_temporal_sensitivity_relative.csv"
  temporal_sensitivity_data = {
    "Feature": [temporal_sensitivity_ft],
    "Accuracy": [correct_percentage],
    "Tmerange": [temporal_sensitivity_timerange],
    "Relative STD Scaler": [temporal_std_scaler]
  }
  temporal_sensitivity_df = pd.DataFrame(temporal_sensitivity_data)
  temporal_sensitivity_df.to_csv(temporal_sensitivity_csv, index = False, mode = "a", header = False)

# Main


In [46]:
def Main(experiment_type, lookback, input_cols, df_train, df_test, model_file, analysis_method:str = None, parameters: list = None):
  base = "/Users/alexandranava/Desktop/Spores/"
  df_train, df_test = initialize_data(df_train, df_test, input_cols)
  trainX, trainY, valX, valY, testX, testY = split_data(df_train, df_test, lookback, input_cols)
  
  models, closest = typical_model(experiment_type, trainX,trainY,valX,valY,testX,testY,range(1), model_file)
  model = models[closest]

  if analysis_method == "local_sensitivity":
    sensitivity_feature = parameters[0]
    scale_factor = parameters[1]
    df_test, delta_x = local_sensitivity_analysis(df_test, sensitivity_feature, scale_factor)
  if analysis_method == "temporal_sensitivity":
    temporal_sensitivity_ft = parameters[0]
    temporal_sensitivity_timerange = parameters[1]
    temporal_std_scaler = parameters[2]
    df_test = temporal_sensitivity_analysis(df_test, temporal_sensitivity_ft, temporal_sensitivity_timerange,  temporal_std_scaler)
  
  trainPredict, valPredict, testPredict = predict_test_data(model, trainX, valX, testX)

  #accuracy_precision_recall_f1score(trainY, trainPredict, valY, valPredict, testY, testPredict)
  df_test = model_prediction_results(model, df_test, input_cols)
  plot_population_prediction(df_test)
  spore_error, actual_germination, predicted_germination = plot_individual_predictions(df_test)
  
  correct_percentage = (sum(spore_error) / len(spore_error)) * 100
  print(f"{correct_percentage}% of spores predicted correctly...")
  number_wrong = spore_error.count(0)
  print(f"{number_wrong} spores incorrectly predicted...")

  if analysis_method == "local_sensitivity":
    write_sensitivity_analysis_results(base, experiment_type, scale_factor, sensitivity_feature, delta_x, correct_percentage)
  if analysis_method == "temporal_sensitivity":
    write_temporal_sensitivity_results(base, experiment_type, temporal_sensitivity_ft, correct_percentage, temporal_sensitivity_timerange, temporal_std_scaler)

Parameters

In [47]:
if __name__ == "__main__":
  #Parameters--
  model_file = "/Users/alexandranava/Desktop/Spores/GerminationPrediction/unmutated_initial_0_intensityscaled.h5"
  df_test = pd.read_csv("/Users/alexandranava/Desktop/Spores/M4581_s1/Analysis/V3/M4581_s1_Model_Data_V2.csv")
  df_train = pd.read_csv("/Users/alexandranava/Desktop/Spores/M4576_s2/M4576_s2_Model_Data_V2.csv")
  input_cols = ['INTENSITY','AREA','GERMINANT_EXPOSURE','ELLIPSE_MINOR', "PERIMETER", 'GERMINATION']
  experiment_type = "unmutated"
  lookback = 3

In [53]:
#Local Sensitivity Analysis

analysis_method = "local_sensitivity"
feature_sets = [["AREA", "PERIMETER", "ELLIPSE_MINOR"], ["INTENSITY"]]
for i in range(4):
  for features in feature_sets:
    for scale_factor in [0.1, 0.5, 1.0]:
      parameters = [features, scale_factor]
      Main(experiment_type, lookback, input_cols, df_train, df_test, model_file, analysis_method, parameters)

training on spores (1, 64)...
validating on spores (65, 76)...
testing on spores ((1, 54))...
loading model...
473/473 [==============================] - 0s 605us/step
Training RMSE: 0.08574929257125442
Validation RMSE: 0.08574929257125442
Testing RMSE: 0.19476770468555774
Model number: 0, distance from mean: 0.0
Returning model 0, whose distance is 0.0
perturbing Area...
Area has std = 63.58600052275756...
perturbing Perimeter...
Perimeter has std = 6.026533669715681...
perturbing Ellipse_Minor...
Ellipse_Minor has std = 0.9657982521567482...
473/473 [==============================] - 0s 606us/step
Writing results for model 1
track 1 of 54...
1/1 [==============================] - 0s 16ms/step
track 2 of 54...
1/1 [==============================] - 0s 13ms/step
track 3 of 54...
1/1 [==============================] - 0s 9ms/step
track 4 of 54...
1/1 [==============================] - 0s 10ms/step
track 5 of 54...
1/1 [==============================] - 0s 9ms/step
track 6 of 54...
1/1 [

Temporal Sensitivity Analysis

In [ ]:
ranges = [[0, 3], [-3, 0]]
# Temporal Sensitivity Analysis
temporal_std_scalers = [1]
temporal_fts = ["PERIMETER", "AREA", "ELLIPSE_MINOR", "INTENSITY"]
analysis_method = "temporal_sensitivity"

for i in range(9):
  for trange in ranges:
    for scaler in temporal_std_scalers:
      
      parameters = [temporal_fts, trange, scaler]
      Main(experiment_type, lookback, input_cols, df_train, df_test, model_file, analysis_method, parameters)

No Analysis

In [ ]:
Main(experiment_type, lookback, input_cols, df_train, df_test, model_file)